In [ ]:
import os
from PIL import Image
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [ ]:
# HYPERPARAMETERS
LEARNING_RATE = 0.001
NUM_EPOCHS = 100
BATCH_SIZE = 100
NOISE_SIZE = 100

Part 1: Create Grumpy Cat Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dir, transform = transforms.ToTensor()):
        self.total_imgs: list[torch.Tensor] = []
        for file in os.listdir(dir):
            self.total_imgs.append( Image.open(dir + '/' + file) )
        self.transform = transform
    
    def __len__(self) -> int:
        return len(self.total_imgs)
    
    def __getitem__(self, idx) -> Image.Image:
        return self.transform(self.total_imgs[idx])

dataset = CustomDataset("grumpifyCat")
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE)

Part 2: Data Augmentation

In [ ]:
def get_transform(mode: str) -> nn.Module:
    if mode == "simple":
        return transforms.Compose([
            transforms.Resize((64, 64), transforms.InterpolationMode.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    elif mode == "deluxe":
        return transforms.Compose([
            transforms.Resize((64, 64), transforms.InterpolationMode.BICUBIC),
            transforms.RandomGrayscale(0.2),
            transforms.RandomRotation(180),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    raise NotImplementedError

Part 3: Implement the Discriminator of the DCGAN

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        # size_out = (size_in + 2 * padding - kernel) / stride + 1
        self.layers = nn.Sequential(
            # INPUT 3x64x64
            nn.Conv2d(  3,  32, kernel_size=4, stride=2, padding=1), # 32x32x32
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.Conv2d( 32,  64, kernel_size=4, stride=2, padding=1), # 64x16x16
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.Conv2d( 64, 128, kernel_size=4, stride=2, padding=1), # 128x8x8
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1), # 256x4x4
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.Conv2d(256, 1, kernel_size=6, stride=2, padding=1), # 1x1x1
        )
    
    def forward(self, z) -> torch.Tensor:
        return self.layers(z).squeeze()

Part 4: Implement the Generator of the DCGAN

In [ ]:
class Generator(nn.Module):
    def __init__(self, noise_size=100):
        super(Generator, self).__init__()
        # size_out = (size_in - 1) * stride - 2 * padding + kernel
        self.layers = nn.Sequential(
            # INPUT nsx1x1
            nn.ConvTranspose2d(noise_size, 256, kernel_size=6, stride=2, padding=1),    # 256x4x4
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),           # 128x8x8
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),            # 64x16x16
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.ConvTranspose2d( 64, 32, kernel_size=4, stride=2, padding=1),            # 32x32x32
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.ConvTranspose2d( 32,  3, kernel_size=4, stride=2, padding=1),            # 3x64x64
            nn.Tanh(),
        )
    
    def forward(self, z) -> torch.Tensor:
        return self.layers(z)

Part 5: Training Loop

In [19]:
def training_loop(dataloader: DataLoader):
    # Create generators and discriminators
    generator = Generator(NOISE_SIZE)
    discriminator = Discriminator()

    # Create optimizers for the generators and discriminators
    g_optimizer = optim.Adam(generator.parameters(), lr=LEARNING_RATE)
    d_optimizer = optim.Adam(discriminator.parameters(), lr=LEARNING_RATE)

    for epoch in range(NUM_EPOCHS):
        iteration = 0
        for data in dataloader:
            n = data.shape[0]
            # 
            # TRAIN DISCRIMINATOR
            # 
            
            d_optimizer.zero_grad()

            # 1. Compute the discriminator loss on real images
            D_real_pred: torch.Tensor = discriminator(data)
            D_real_loss: torch.Tensor = (1 - D_real_pred).pow(2)

            # 2. Sample noise
            noise: torch.Tensor = torch.rand((n,NOISE_SIZE,1,1))

            # 3. Generate fake images from the noise
            fake_images: torch.Tensor = generator(noise)

            # 4. Compute the discriminator loss on the fake images
            D_fake_pred: torch.Tensor = discriminator(fake_images)
            D_fake_loss: torch.Tensor = (-1 - D_fake_pred).pow(2)

            # 5. Compute total loss
            D_total_loss: torch.Tensor = (D_real_loss + D_fake_loss).sum().div(n)
            if iteration % 2 == 0:
                D_total_loss.backward()
                d_optimizer.step()

            # 
            # TRAIN GENERATOR
            # 

            g_optimizer.zero_grad()

            # 1. Sample noise
            noise: torch.Tensor = torch.rand((n,NOISE_SIZE,1,1))

            # 2. Generate fake images from the noise
            fake_images: torch.Tensor = generator(noise)

            # 3. Compute the generator loss
            D_gen_pred: torch.Tensor = discriminator(fake_images)
            G_loss: torch.Tensor = (1 - D_gen_pred).pow(2).sum().div(n)

            G_loss.backward()
            g_optimizer.step()

            # Print each loss every 200 iterations
            if iteration % 200 == 0:
                print(f"Loss - Discriminator: {D_total_loss:<8.2f} Generator: {G_loss:<8.2f}")

training_loop(dataloader)

Loss - Discriminator: 2.32     Generator: 28.79   
Loss - Discriminator: 22.05    Generator: 42.72   
Loss - Discriminator: 1.87     Generator: 38.67   
Loss - Discriminator: 5.96     Generator: 3.95    
Loss - Discriminator: 8.49     Generator: 12.82   
Loss - Discriminator: 0.98     Generator: 21.01   
Loss - Discriminator: 0.76     Generator: 11.69   
Loss - Discriminator: 0.86     Generator: 8.49    
Loss - Discriminator: 0.70     Generator: 12.40   
Loss - Discriminator: 0.37     Generator: 13.39   
Loss - Discriminator: 0.51     Generator: 10.81   
Loss - Discriminator: 0.79     Generator: 10.11   
Loss - Discriminator: 0.50     Generator: 11.47   
Loss - Discriminator: 0.54     Generator: 10.95   
Loss - Discriminator: 0.21     Generator: 9.91    
Loss - Discriminator: 0.34     Generator: 10.63   
Loss - Discriminator: 0.32     Generator: 11.72   
Loss - Discriminator: 0.11     Generator: 12.84   
Loss - Discriminator: 0.22     Generator: 12.27   
Loss - Discriminator: 0.31     

KeyboardInterrupt: 